In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configuración del WebDriver (en este caso, Chrome) para modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en modo headless (sin interfaz gráfica)

# Configura el path del ChromeDriver
service = Service(r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\SELENIUM\chromedriver.exe')

# Inicia el navegador con la configuración headless
driver = webdriver.Chrome(service=service, options=chrome_options)

# Lista para almacenar los productos extraídos
productos_extraidos = []

# Función para extraer la información de los productos en una página
def extraer_productos(url):
    driver.get(url)
    
    # Esperar hasta que los resultados de búsqueda estén completamente cargados
    try:
        # Esperar hasta que los resultados de productos estén visibles
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#testId-searchResults-products > div"))
        )
        
        # Buscar los productos
        productos = driver.find_elements(By.CSS_SELECTOR, "#testId-searchResults-products > div")
        
        if len(productos) == 0:
            print("No se encontraron productos en esta página. Finalizando el scraping.")
            return False
        
        # Iterar sobre los productos y extraer los datos
        for producto in productos:
            try:
                # Variable para almacenar la URL de la imagen
                url_imagen = ""

                # Extraer la URL de la imagen (si está disponible)
                try:
                    img_url = producto.find_element(By.CSS_SELECTOR, "img").get_attribute("src")
                    # Suponiendo que ya tienes una variable 'img_url' que contiene la URL de la imagen
                    # Aquí va el código para verificar si la URL termina en "fit=pad":
                    if img_url and img_url.endswith("fit=pad"):
                        url_imagen = img_url
                except:
                    url_imagen = ""
                
                # Extraer el nombre del producto
                nombre_producto = producto.find_element(By.CSS_SELECTOR, "b.pod-subTitle.subTitle-rebrand").text.strip()

                # Extraer el precio original (si existe)
                try:
                    precio_original = producto.find_element(By.CSS_SELECTOR, "li[data-normal-price]").get_attribute('data-normal-price')
                except:
                    precio_original = ""

                # Extraer el precio final (si existe)
                try:
                    precio_final = producto.find_element(By.CSS_SELECTOR, "li[data-internet-price]").get_attribute('data-internet-price')
                except:
                    precio_final = ""

                # Extraer la URL del producto
                url_producto = producto.find_element(By.CSS_SELECTOR, "a").get_attribute('href')

                # Guardar los datos en la lista
                productos_extraidos.append({
                    "Producto": nombre_producto,
                    "Precio original": precio_original,
                    "Precio final": precio_final,
                    "URL del producto": url_producto,
                    "URL de la imagen": url_imagen
                })

                # Imprimir los resultados
                print(f"Producto: {nombre_producto}")
                print(f"Precio original: {precio_original}")
                print(f"Precio final: {precio_final}")
                print(f"URL del producto: {url_producto}")
                print(f"URL de la imagen: {url_imagen}")
                print("-" * 50)

            except Exception as e:
                print(f"Error al extraer datos del producto: {e}")
        
        return True  # Retorna True si se encontraron productos

    except Exception as e:
        print(f"Error al cargar la página: {e}")
        return False

# Número de páginas a recorrer
pagina = 1
continuar = True

while continuar:
    print(f"Accediendo a la página {pagina}...")
    
    # URL de la página actual
    url_pagina = f"https://www.falabella.com/falabella-cl/brand/YAMP?page={pagina}"
    
    # Extraer productos de la página actual
    continuar = extraer_productos(url_pagina)
    
    # Si se encontraron productos, pasar a la siguiente página
    if continuar:
        pagina += 1
        time.sleep(10)  # Espera de 10 segundos entre páginas
    else:
        print("No hay más productos o hubo un error. Terminando el scraping.")
        break

# Cerrar el navegador
driver.quit()


Accediendo a la página 1...
Producto: Packs 2 Calzas Niña Algodón
Precio original: 
Precio final: 7.990
URL del producto: https://www.falabella.com/falabella-cl/product/883161284/Packs-2-Calzas-Nina-Algodon-Yamp/883161303
URL de la imagen: https://www.falabella.com/cdn-cgi/imagedelivery/4fYuQyy-r8_rpBpcY7lH_A/falabellaCL/883161303/width=240,height=240,quality=70,format=webp,fit=pad
--------------------------------------------------
Producto: Sandalia Niña Plateado (24 a 37)
Precio original: 
Precio final: 14.990
URL del producto: https://www.falabella.com/falabella-cl/product/883204528/Sandalia-Nina-Plateado-(24-a-37)-Yamp/883204536
URL de la imagen: https://www.falabella.com/cdn-cgi/imagedelivery/4fYuQyy-r8_rpBpcY7lH_A/falabellaCL/883204536/width=240,height=240,quality=70,format=webp,fit=pad
--------------------------------------------------
Producto: Vestido Niña
Precio original: 
Precio final: 18.990
URL del producto: https://www.falabella.com/falabella-cl/product/883230406/Vestido-


KeyboardInterrupt



In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configuración del WebDriver (en este caso, Chrome) para modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en modo headless (sin interfaz gráfica)

# Configura el path del ChromeDriver
service = Service(r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\SELENIUM\chromedriver.exe')

# Inicia el navegador con la configuración headless
driver = webdriver.Chrome(service=service, options=chrome_options)

# Ruta del archivo Excel donde se almacenarán los productos
ruta_excel = r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\EN5.xlsx"

# Cargar el archivo Excel existente (si existe)
try:
    df_existente = pd.read_excel(ruta_excel)
    print("Archivo Excel cargado con éxito.")
except FileNotFoundError:
    df_existente = pd.DataFrame(columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])
    print("No se encontró el archivo Excel. Se creará uno nuevo.")

# Lista para almacenar los productos extraídos
productos_extraidos = []

# Función para extraer la información de los productos en una página
def extraer_productos(url):
    driver.get(url)
    
    # Esperar hasta que los resultados de búsqueda estén completamente cargados
    try:
        # Esperar hasta que los resultados de productos estén visibles
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#testId-searchResults-products > div"))
        )
        
        # Buscar los productos
        productos = driver.find_elements(By.CSS_SELECTOR, "#testId-searchResults-products > div")
        
        if len(productos) == 0:
            print("No se encontraron productos en esta página. Finalizando el scraping.")
            return False
        
        # Iterar sobre los productos y extraer los datos
        for producto in productos:
            try:
                # Variable para almacenar la URL de la imagen
                url_imagen = ""

                # Extraer la URL de la imagen (si está disponible)
                try:
                    img_url = producto.find_element(By.CSS_SELECTOR, "img").get_attribute("src")
                    if img_url and img_url.endswith("fit=pad"):
                        url_imagen = img_url
                except:
                    url_imagen = ""
                
                # Extraer el nombre del producto
                nombre_producto = producto.find_element(By.CSS_SELECTOR, "b.pod-subTitle.subTitle-rebrand").text.strip()

                # Extraer el precio original (si existe)
                try:
                    precio_original = producto.find_element(By.CSS_SELECTOR, "li[data-normal-price]").get_attribute('data-normal-price')
                except:
                    precio_original = ""

                # Extraer el precio final (si existe)
                try:
                    precio_final = producto.find_element(By.CSS_SELECTOR, "li[data-internet-price]").get_attribute('data-internet-price')
                except:
                    precio_final = ""

                # Extraer la URL del producto
                url_producto = producto.find_element(By.CSS_SELECTOR, "a").get_attribute('href')

                # Extraer el SKU y la marca si es posible
                sku = ""  # Asignar valor predeterminado
                marca = ""  # Asignar valor predeterminado

                # Guardar los datos en la lista
                productos_extraidos.append({
                    "Nombre del Producto": nombre_producto,
                    "Precio Original": precio_original,
                    "Precio Final": precio_final,
                    "URL del Producto": url_producto,
                    "SKU": sku,
                    "URL de la Imagen": url_imagen,
                    "Marca": marca
                })

                # Imprimir los resultados
                print(f"Producto: {nombre_producto}")
                print(f"Precio original: {precio_original}")
                print(f"Precio final: {precio_final}")
                print(f"URL del producto: {url_producto}")
                print(f"URL de la imagen: {url_imagen}")
                print("-" * 50)

            except Exception as e:
                print(f"Error al extraer datos del producto: {e}")
        
        return True  # Retorna True si se encontraron productos

    except Exception as e:
        print(f"Error al cargar la página: {e}")
        return False

# Número de páginas a recorrer
pagina = 1
continuar = True

while continuar:
    print(f"Accediendo a la página {pagina}...")
    
    # URL de la página actual
    url_pagina = f"https://www.falabella.com/falabella-cl/brand/HOLLEY%20SKOOL?page={pagina}"
    
    # Extraer productos de la página actual
    continuar = extraer_productos(url_pagina)
    
    # Si se encontraron productos, pasar a la siguiente página
    if continuar:
        pagina += 1
        time.sleep(10)  # Espera de 10 segundos entre páginas
    else:
        print("No hay más productos o hubo un error. Terminando el scraping.")
        break

# Crear el DataFrame final con todos los productos
df_final = pd.DataFrame(productos_extraidos, columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])

# Concatenar los productos extraídos con los productos existentes en el archivo Excel
df_actualizado = pd.concat([df_existente, df_final], ignore_index=True)

# Guardar el DataFrame final en el archivo Excel
df_actualizado.to_excel(ruta_excel, index=False)

# Cerrar el navegador
driver.quit()


Archivo Excel cargado con éxito.
Accediendo a la página 1...
Producto: Zapatilla Escolar Unisex Niño Blanco (34 a 38)
Precio original: 
Precio final: 12.990
URL del producto: https://www.falabella.com/falabella-cl/product/882995205/Zapatilla-Escolar-Unisex-Nino-Blanco-(34-a-38)-Holley-Skool/882995210
URL de la imagen: https://www.falabella.com/cdn-cgi/imagedelivery/4fYuQyy-r8_rpBpcY7lH_A/falabellaCL/882995210_1/width=240,height=240,quality=70,format=webp,fit=pad
--------------------------------------------------
Producto: Set 1 Mini Mochila, 1 Destacador, 1 Boligrafo, 1 Mini Libreta , 1 Sticker Unisex
Precio original: 
Precio final: 5.990
URL del producto: https://www.falabella.com/falabella-cl/product/882673624/Set-1-Mini-Mochila,-1-Destacador,-1-Boligrafo,-1-Mini-Libreta-,-1-Sticker-Unisex-Holley-Skool/882673625
URL de la imagen: https://www.falabella.com/cdn-cgi/imagedelivery/4fYuQyy-r8_rpBpcY7lH_A/falabellaCL/882673625_1/width=240,height=240,quality=70,format=webp,fit=pad
---------

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configuración del WebDriver (en este caso, Chrome) para modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en modo headless (sin interfaz gráfica)

# Configura el path del ChromeDriver
service = Service(r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\SELENIUM\chromedriver.exe')

# Inicia el navegador con la configuración headless
driver = webdriver.Chrome(service=service, options=chrome_options)

# Ruta del archivo Excel donde se almacenarán los productos
ruta_excel = r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\YUYA.xlsx"

# Cargar el archivo Excel existente (si existe)
try:
    df_existente = pd.read_excel(ruta_excel)
    print("Archivo Excel cargado con éxito.")
except FileNotFoundError:
    df_existente = pd.DataFrame(columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])
    print("No se encontró el archivo Excel. Se creará uno nuevo.")

# Lista para almacenar los productos extraídos
productos_extraidos = []

# Función para extraer la información de los productos en una página
def extraer_productos(url):
    driver.get(url)
    
    # Esperar hasta que los resultados de búsqueda estén completamente cargados
    try:
        # Esperar hasta que los resultados de productos estén visibles
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#testId-searchResults-products > div"))
        )
        
        # Buscar los productos
        productos = driver.find_elements(By.CSS_SELECTOR, "#testId-searchResults-products > div")
        
        if len(productos) == 0:
            print("No se encontraron productos en esta página. Finalizando el scraping.")
            return False
        
        # Iterar sobre los productos y extraer los datos
        for producto in productos:
            try:
                # Variable para almacenar la URL de la imagen
                url_imagen = ""

                # Extraer la URL de la imagen (si está disponible)
                try:
                    img_url = producto.find_element(By.CSS_SELECTOR, "img").get_attribute("src")
                    if img_url and img_url.endswith("fit=pad"):
                        url_imagen = img_url
                except:
                    url_imagen = ""
                
                # Extraer el nombre del producto
                nombre_producto = producto.find_element(By.CSS_SELECTOR, "b.pod-subTitle.subTitle-rebrand").text.strip()

                # Extraer el precio original (si existe)
                try:
                    precio_original = producto.find_element(By.CSS_SELECTOR, "li[data-normal-price]").get_attribute('data-normal-price')
                except:
                    precio_original = ""

                # Extraer el precio final (si existe)
                try:
                    precio_final = producto.find_element(By.CSS_SELECTOR, "li[data-internet-price]").get_attribute('data-internet-price')
                except:
                    precio_final = ""

                # Extraer la URL del producto
                url_producto = producto.find_element(By.CSS_SELECTOR, "a").get_attribute('href')

                # Extraer el SKU y la marca si es posible
                sku = ""  # Asignar valor predeterminado
                marca = ""  # Asignar valor predeterminado

                # Guardar los datos en la lista
                productos_extraidos.append({
                    "Nombre del Producto": nombre_producto,
                    "Precio Original": precio_original,
                    "Precio Final": precio_final,
                    "URL del Producto": url_producto,
                    "SKU": sku,
                    "URL de la Imagen": url_imagen,
                    "Marca": marca
                })

                # Imprimir los resultados
                print(f"Producto: {nombre_producto}")
                print(f"Precio original: {precio_original}")
                print(f"Precio final: {precio_final}")
                print(f"URL del producto: {url_producto}")
                print(f"URL de la imagen: {url_imagen}")
                print("-" * 50)

            except Exception as e:
                print(f"Error al extraer datos del producto: {e}")
        
        return True  # Retorna True si se encontraron productos

    except Exception as e:
        print(f"Error al cargar la página: {e}")
        return False

# Número de páginas a recorrer
pagina = 1
continuar = True

while continuar:
    print(f"Accediendo a la página {pagina}...")
    
    # URL de la página actual
    url_pagina = f"https://www.falabella.com/falabella-cl/brand/YAMP?page={pagina}"
    
    # Extraer productos de la página actual
    continuar = extraer_productos(url_pagina)
    
    # Si se encontraron productos, pasar a la siguiente página
    if continuar:
        pagina += 1
        time.sleep(10)  # Espera de 10 segundos entre páginas
    else:
        print("No hay más productos o hubo un error. Terminando el scraping.")
        break

# Crear el DataFrame final con todos los productos
df_final = pd.DataFrame(productos_extraidos, columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])

# Concatenar los productos extraídos con los productos existentes en el archivo Excel
df_actualizado = pd.concat([df_existente, df_final], ignore_index=True)

# Guardar el DataFrame final en el archivo Excel
df_actualizado.to_excel(ruta_excel, index=False)

# Cerrar el navegador
driver.quit()


Archivo Excel cargado con éxito.
Accediendo a la página 1...
Producto: Zapatilla Casual Niña Rosado (22 a 36)
Precio original: 
Precio final: 14.990
URL del producto: https://www.falabella.com/falabella-cl/product/883149719/Zapatilla-Casual-Nina-Rosado-(22-a-36)-Yamp/883149722
URL de la imagen: https://www.falabella.com/cdn-cgi/imagedelivery/4fYuQyy-r8_rpBpcY7lH_A/falabellaCL/883149722_1/width=240,height=240,quality=70,format=webp,fit=pad
--------------------------------------------------
Producto: Short Algodón Niño
Precio original: 12.990
Precio final: 9.990
URL del producto: https://www.falabella.com/falabella-cl/product/883233447/Short-Algodon-Nino-Yamp/883233451
URL de la imagen: https://www.falabella.com/cdn-cgi/imagedelivery/4fYuQyy-r8_rpBpcY7lH_A/falabellaCL/883233451_1/width=240,height=240,quality=70,format=webp,fit=pad
--------------------------------------------------
Producto: Pack 3 Pantalones De Buzo Niño
Precio original: 
Precio final: 22.990
URL del producto: https://ww

In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configuración del WebDriver (en este caso, Chrome) para modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en modo headless (sin interfaz gráfica)

# Configura el path del ChromeDriver
service = Service(r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\SELENIUM\chromedriver.exe')

# Inicia el navegador con la configuración headless
driver = webdriver.Chrome(service=service, options=chrome_options)

# Ruta del archivo Excel donde se almacenarán los productos
ruta_excel = r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\ESCOLAR3.xlsx"

# Cargar el archivo Excel existente (si existe)
try:
    df_existente = pd.read_excel(ruta_excel)
    print("Archivo Excel cargado con éxito.")
except FileNotFoundError:
    df_existente = pd.DataFrame(columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])
    print("No se encontró el archivo Excel. Se creará uno nuevo.")

# Lista para almacenar los productos extraídos
productos_extraidos = []

# Función para extraer la información de los productos en una página
def extraer_productos(url):
    driver.get(url)
    
    # Esperar hasta que los resultados de búsqueda estén completamente cargados
    try:
        # Esperar hasta que los resultados de productos estén visibles
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#testId-searchResults-products > div"))
        )
        
        # Buscar los productos
        productos = driver.find_elements(By.CSS_SELECTOR, "#testId-searchResults-products > div")
        
        if len(productos) == 0:
            print("No se encontraron productos en esta página. Finalizando el scraping.")
            return False
        
        # Iterar sobre los productos y extraer los datos
        for producto in productos:
            try:
                # Variable para almacenar la URL de la imagen
                url_imagen = ""

                # Extraer la URL de la imagen (si está disponible)
                try:
                    img_url = producto.find_element(By.CSS_SELECTOR, "img").get_attribute("src")
                    if img_url and img_url.endswith("fit=pad"):
                        url_imagen = img_url
                except:
                    url_imagen = ""
                
                # Extraer el nombre del producto
                nombre_producto = producto.find_element(By.CSS_SELECTOR, "b.pod-subTitle.subTitle-rebrand").text.strip()

                # Extraer el precio original (si existe)
                try:
                    precio_original = producto.find_element(By.CSS_SELECTOR, "li[data-normal-price]").get_attribute('data-normal-price')
                except:
                    precio_original = ""

                # Extraer el precio final (si existe)
                try:
                    precio_final = producto.find_element(By.CSS_SELECTOR, "li[data-internet-price]").get_attribute('data-internet-price')
                except:
                    precio_final = ""

                # Extraer la URL del producto
                url_producto = producto.find_element(By.CSS_SELECTOR, "a").get_attribute('href')

                # Extraer el SKU y la marca si es posible
                sku = ""  # Asignar valor predeterminado

                # Extraer la marca
                try:
                    marca = producto.find_element(By.CSS_SELECTOR, "b.jsx-184544934.title5.secondary.jsx-3451706699.bold.pod-title.title-rebrand").text.strip()
                except:
                    marca = ""  # Si no se encuentra la marca, asigna un valor vacío
                
                # Guardar los datos en la lista
                productos_extraidos.append({
                    "Nombre del Producto": nombre_producto,
                    "Precio Original": precio_original,
                    "Precio Final": precio_final,
                    "URL del Producto": url_producto,
                    "SKU": sku,
                    "URL de la Imagen": url_imagen,
                    "Marca": marca
                })

                # Imprimir los resultados
                print(f"Producto: {nombre_producto}")
                print(f"Precio original: {precio_original}")
                print(f"Precio final: {precio_final}")
                print(f"URL del producto: {url_producto}")
                print(f"URL de la imagen: {url_imagen}")
                print(f"Marca: {marca}")
                print("-" * 50)

            except Exception as e:
                print(f"Error al extraer datos del producto: {e}")
        
        return True  # Retorna True si se encontraron productos

    except Exception as e:
        print(f"Error al cargar la página: {e}")
        return False

# Número de páginas a recorrer
pagina = 1
continuar = True

while continuar:
    print(f"Accediendo a la página {pagina}...")
    
    # URL de la página actual
    url_pagina = f"https://www.falabella.com/falabella-cl/category/cat1530002/Escolares?page={pagina}"
    
    # Extraer productos de la página actual
    continuar = extraer_productos(url_pagina)
    
    # Si se encontraron productos, pasar a la siguiente página
    if continuar:
        pagina += 1
        time.sleep(10)  # Espera de 10 segundos entre páginas
    else:
        print("No hay más productos o hubo un error. Terminando el scraping.")
        break

# Crear el DataFrame final con todos los productos
df_final = pd.DataFrame(productos_extraidos, columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])

# Concatenar los productos extraídos con los productos existentes en el archivo Excel
df_actualizado = pd.concat([df_existente, df_final], ignore_index=True)

# Guardar el DataFrame final en el archivo Excel
df_actualizado.to_excel(ruta_excel, index=False)

# Cerrar el navegador
driver.quit()


Archivo Excel cargado con éxito.
Accediendo a la página 1...
Producto: Lonchera Eléctrica Portátil Calentador De Alimentos 2 En 1
Precio original: 23.990
Precio final: 
URL del producto: https://www.falabella.com/falabella-cl/product/123532439/Lonchera-Electrica-Portatil-Calentador-De-Alimentos-2-En-1/123532440?sponsoredClickData=%257B%2522isXLP%2522%253Atrue%252C%2522pageType%2522%253A%2522category%2522%252C%2522adType%2522%253A%2522products%2522%252C%2522platform%2522%253A%2522desktop%2522%252C%2522regionCode%2522%253A%2522cl%2522%252C%2522context%2522%253A%257B%2522userIdentifier%2522%253A%2522MTczMzY2NjY4Mjg0Nw%253D%253D%2522%252C%2522adIdentifier%2522%253A%2522v2_D2u6rJjdN0PHBNQaI_fGJrRcpu4zZMhOlJcET_dFfutq93Q7z84UmSQZb8XLPtw8I0utASu6TsSoSyKb_8GXRXBCXIaduGnNU_jW-ATlK3U80vUPHmZtDCSLWyHLBwSFg-1fCcemmPOmLiDIZGHoA6UZ4VyvFYaG4hNCR75NG5SAySKSgKvMNDcRi-XzK-jz_ark193a697c4fb10a63db%2522%252C%2522slot%2522%253A1%252C%2522signal%2522%253A%2522Escolares%2522%257D%257D
URL de la imagen: https

Exception ignored in: <function ZipFile.__del__ at 0x00000173103BB920>
Traceback (most recent call last):
  File "C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\zipfile\__init__.py", line 1940, in __del__
    self.close()
  File "C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\zipfile\__init__.py", line 1942, in close
    def close(self):

KeyboardInterrupt: 

KeyboardInterrupt



In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configuración del WebDriver (en este caso, Chrome) para modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en modo headless (sin interfaz gráfica)

# Configura el path del ChromeDriver
service = Service(r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\SELENIUM\chromedriver.exe')

# Inicia el navegador con la configuración headless
driver = webdriver.Chrome(service=service, options=chrome_options)

# Ruta del archivo Excel donde se almacenarán los productos
ruta_excel = r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\FALO3.xlsx"

# Cargar el archivo Excel existente (si existe)
try:
    df_existente = pd.read_excel(ruta_excel)
    print("Archivo Excel cargado con éxito.")
except FileNotFoundError:
    df_existente = pd.DataFrame(columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])
    print("No se encontró el archivo Excel. Se creará uno nuevo.")

# Lista para almacenar los productos extraídos
productos_extraidos = []

# Función para extraer la información de los productos en una página
def extraer_productos(url):
    driver.get(url)
    
    # Esperar hasta que los resultados de búsqueda estén completamente cargados
    try:
        # Esperar hasta que los resultados de productos estén visibles
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#testId-searchResults-products > div"))
        )
        
        # Buscar los productos
        productos = driver.find_elements(By.CSS_SELECTOR, "#testId-searchResults-products > div")
        
        if len(productos) == 0:
            print("No se encontraron productos en esta página. Finalizando el scraping.")
            return False
        
        # Iterar sobre los productos y extraer los datos
        for producto in productos:
            try:
                # Variable para almacenar la URL de la imagen
                url_imagen = ""

                # Extraer la URL de la imagen (si está disponible)
                try:
                    img_url = producto.find_element(By.CSS_SELECTOR, "img").get_attribute("src")
                    if img_url and img_url.endswith("fit=pad"):
                        url_imagen = img_url
                except:
                    url_imagen = ""
                
                # Extraer el nombre del producto
                nombre_producto = producto.find_element(By.CSS_SELECTOR, "b.pod-subTitle.subTitle-rebrand").text.strip()

                # Extraer el precio original (si existe)
                try:
                    precio_original = producto.find_element(By.CSS_SELECTOR, "li[data-normal-price]").get_attribute('data-normal-price')
                except:
                    precio_original = ""

                # Extraer el precio final (si existe)
                try:
                    precio_final = producto.find_element(By.CSS_SELECTOR, "li[data-internet-price]").get_attribute('data-internet-price')
                except:
                    precio_final = ""

                # Extraer la URL del producto
                url_producto = producto.find_element(By.CSS_SELECTOR, "a").get_attribute('href')

                # Extraer el SKU
                sku = ""  # Asignar valor predeterminado

                # Extraer la marca, que está en el texto de la etiqueta <b> con id 'testId-pod-displaySellerText'
                try:
                    marca_texto = producto.find_element(By.CSS_SELECTOR, "b[id^='testId-pod-displaySellerText']").text
                    if marca_texto.startswith("Por "):
                        marca = marca_texto[3:].strip()  # Eliminar el "Por " y obtener el nombre de la marca
                    else:
                        marca = ""  # Si no tiene "Por", no extraemos nada
                except:
                    marca = ""  # Si no se encuentra la marca, asigna un valor vacío
                
                # Guardar los datos en la lista
                productos_extraidos.append({
                    "Nombre del Producto": nombre_producto,
                    "Precio Original": precio_original,
                    "Precio Final": precio_final,
                    "URL del Producto": url_producto,
                    "SKU": sku,
                    "URL de la Imagen": url_imagen,
                    "Marca": marca
                })

                # Imprimir los resultados
                print(f"Producto: {nombre_producto}")
                print(f"Precio original: {precio_original}")
                print(f"Precio final: {precio_final}")
                print(f"URL del producto: {url_producto}")
                print(f"URL de la imagen: {url_imagen}")
                print(f"Marca: {marca}")
                print("-" * 50)

            except Exception as e:
                print(f"Error al extraer datos del producto: {e}")
        
        return True  # Retorna True si se encontraron productos

    except Exception as e:
        print(f"Error al cargar la página: {e}")
        return False

# Número de páginas a recorrer
pagina = 1
continuar = True

while continuar:
    print(f"Accediendo a la página {pagina}...")
    
    # URL de la página actual
    url_pagina = f"https://www.falabella.com/falabella-cl/category/cat1530008/Zapatos-Escolares?page={pagina}"
    
    # Extraer productos de la página actual
    continuar = extraer_productos(url_pagina)
    
    # Si se encontraron productos, pasar a la siguiente página
    if continuar:
        pagina += 1
        time.sleep(10)  # Espera de 10 segundos entre páginas
    else:
        print("No hay más productos o hubo un error. Terminando el scraping.")
        break

# Crear el DataFrame final con todos los productos
df_final = pd.DataFrame(productos_extraidos, columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])

# Concatenar los productos extraídos con los productos existentes en el archivo Excel
df_actualizado = pd.concat([df_existente, df_final], ignore_index=True)

# Guardar el DataFrame final en el archivo Excel
df_actualizado.to_excel(ruta_excel, index=False)

# Cerrar el navegador
driver.quit()


Archivo Excel cargado con éxito.
Accediendo a la página 1...
Producto: Calzado Escolar Niña Negro (28 a 34)
Precio original: 
Precio final: 59.990
URL del producto: https://www.falabella.com/falabella-cl/product/6025322/Calzado-Escolar-Nina-Negro-(28-a-34)-Hush-Puppies/6025328
URL de la imagen: https://www.falabella.com/cdn-cgi/imagedelivery/4fYuQyy-r8_rpBpcY7lH_A/falabellaCL/6025328_1/width=240,height=240,quality=70,format=webp,fit=pad
Marca: Falabella
--------------------------------------------------
Producto: Zapato Escolar Niño Full Negro Muzat Unisport
Precio original: 47.990
Precio final: 
URL del producto: https://www.falabella.com/falabella-cl/product/124103731/Zapato-Escolar-Nino-Full-Negro-Muzat-Unisport/124103740
URL de la imagen: https://www.falabella.com/cdn-cgi/imagedelivery/4fYuQyy-r8_rpBpcY7lH_A/falabellaCL/124103740_01/width=240,height=240,quality=70,format=webp,fit=pad
Marca: Komo2
--------------------------------------------------
Producto: Court Borough Low 2 Zapat

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configuración del WebDriver (en este caso, Chrome) para modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en modo headless (sin interfaz gráfica)

# Configura el path del ChromeDriver
service = Service(r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\SELENIUM\chromedriver.exe')

# Inicia el navegador con la configuración headless
driver = webdriver.Chrome(service=service, options=chrome_options)

# Ruta del archivo Excel donde se almacenarán los productos
ruta_excel = r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\FA1.xlsx"

# Cargar el archivo Excel existente (si existe)
try:
    df_existente = pd.read_excel(ruta_excel)
    print("Archivo Excel cargado con éxito.")
except FileNotFoundError:
    df_existente = pd.DataFrame(columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])
    print("No se encontró el archivo Excel. Se creará uno nuevo.")

# Lista para almacenar los productos extraídos
productos_extraidos = []

# Función para extraer la información de los productos en una página
def extraer_productos(url):
    driver.get(url)
    
    # Esperar hasta que los resultados de búsqueda estén completamente cargados
    try:
        # Esperar hasta que los resultados de productos estén visibles
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#testId-searchResults-products > div"))
        )
        
        # Buscar los productos
        productos = driver.find_elements(By.CSS_SELECTOR, "#testId-searchResults-products > div")
        
        if len(productos) == 0:
            print("No se encontraron productos en esta página. Finalizando el scraping.")
            return False
        
        # Iterar sobre los productos y extraer los datos
        for producto in productos:
            try:
                # Variable para almacenar la URL de la imagen
                url_imagen = ""

                # Extraer la URL de la imagen (si está disponible)
                try:
                    img_url = producto.find_element(By.CSS_SELECTOR, "img").get_attribute("src")
                    if img_url and img_url.endswith("fit=pad"):
                        url_imagen = img_url
                except:
                    url_imagen = ""
                
                # Extraer el nombre del producto
                nombre_producto = producto.find_element(By.CSS_SELECTOR, "b.pod-subTitle.subTitle-rebrand").text.strip()

                # Extraer el precio original (si existe)
                try:
                    precio_original = producto.find_element(By.CSS_SELECTOR, "li[data-normal-price]").get_attribute('data-normal-price')
                except:
                    precio_original = ""

                # Extraer el precio final (si existe)
                try:
                    precio_final = producto.find_element(By.CSS_SELECTOR, "li[data-internet-price]").get_attribute('data-internet-price')
                except:
                    precio_final = ""

                # Extraer la URL del producto
                url_producto = producto.find_element(By.CSS_SELECTOR, "a").get_attribute('href')

                # Extraer el SKU
                sku = ""  # Asignar valor predeterminado

                # Extraer la marca, que está en el texto de la etiqueta <b> con id 'testId-pod-displaySellerText'
                try:
                    marca_texto = producto.find_element(By.CSS_SELECTOR, "b[id^='testId-pod-displaySellerText']").text
                    if marca_texto.startswith("Por "):
                        marca = marca_texto[3:].strip()  # Eliminar el "Por " y obtener el nombre de la marca
                    else:
                        marca = ""  # Si no tiene "Por", no extraemos nada
                except:
                    marca = ""  # Si no se encuentra la marca, asigna un valor vacío
                
                # Guardar los datos en la lista
                productos_extraidos.append({
                    "Nombre del Producto": nombre_producto,
                    "Precio Original": precio_original,
                    "Precio Final": precio_final,
                    "URL del Producto": url_producto,
                    "SKU": sku,
                    "URL de la Imagen": url_imagen,
                    "Marca": marca
                })

                # Imprimir los resultados
                print(f"Producto: {nombre_producto}")
                print(f"Precio original: {precio_original}")
                print(f"Precio final: {precio_final}")
                print(f"URL del producto: {url_producto}")
                print(f"URL de la imagen: {url_imagen}")
                print(f"Marca: {marca}")
                print("-" * 50)

            except Exception as e:
                print(f"Error al extraer datos del producto: {e}")
        
        return True  # Retorna True si se encontraron productos

    except Exception as e:
        print(f"Error al cargar la página: {e}")
        return False

# Número de páginas a recorrer
pagina = 1
continuar = True

while continuar:
    print(f"Accediendo a la página {pagina}...")
    
    # URL de la página actual
    url_pagina = f"https://www.falabella.com/falabella-cl/category/cat5620004/Moda-Infantil?page={pagina}"
    
    # Extraer productos de la página actual
    continuar = extraer_productos(url_pagina)
    
    # Si se encontraron productos, pasar a la siguiente página
    if continuar:
        pagina += 1
        time.sleep(10)  # Espera de 10 segundos entre páginas
    else:
        print("No hay más productos o hubo un error. Terminando el scraping.")
        break

# Crear el DataFrame final con todos los productos
df_final = pd.DataFrame(productos_extraidos, columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])

# Concatenar los productos extraídos con los productos existentes en el archivo Excel
df_actualizado = pd.concat([df_existente, df_final], ignore_index=True)

# Guardar el DataFrame final en el archivo Excel
df_actualizado.to_excel(ruta_excel, index=False)

# Cerrar el navegador
driver.quit()


Archivo Excel cargado con éxito.
Accediendo a la página 1...
Producto: Camiseta Cuello Polo Manga Corta Blanco
Precio original: 6.990
Precio final: 
URL del producto: https://www.falabella.com/falabella-cl/product/114471700/Camiseta-Cuello-Polo-Manga-Corta-Blanco-Tais/114471702?sponsoredClickData=%257B%2522isXLP%2522%253Atrue%252C%2522pageType%2522%253A%2522category%2522%252C%2522adType%2522%253A%2522products%2522%252C%2522platform%2522%253A%2522desktop%2522%252C%2522regionCode%2522%253A%2522cl%2522%252C%2522context%2522%253A%257B%2522userIdentifier%2522%253A%2522MTczNDI5NTIwNjk4Mw%253D%253D%2522%252C%2522adIdentifier%2522%253A%2522v2_IZSo6cGBpNRKGMF9pgusUD-ZxirBTsJrC0IQ8h3KyTfNQxl2RmQapapRwAGtrE5lqMJwrxj18fqABSdWi0mY3j65zLd_Ysy0SZMYSD5XK8facVhcksSVisQMQMDWWS-g361D8e_o2Q2ZfiNrhaDIXIQN80sPNzkKtA083E4DAmxvQRTe6JSYLcAXFbx7FkAO_ark193cc0e481078f1b31%2522%252C%2522slot%2522%253A1%252C%2522signal%2522%253A%2522Moda-Infantil%2522%257D%257D
URL de la imagen: https://www.falabella.com/cdn-cgi/i